# Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Instaling dependencies

In [1]:
!conda install --yes --quiet pandas seaborn matplotlib scikit-learn pyarrow

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



## Import libs

In [2]:
import pandas as pd

import pickle

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

import numpy as np

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

In [3]:
!mkdir -p data
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o data/yellow_tripdata_2023-01.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -o data/yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  19.2M      0  0:00:02  0:00:02 --:--:-- 19.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0  21.2M      0  0:00:02  0:00:02 --:--:-- 21.2M


In [4]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

Read the data for January. How many columns are there?

In [5]:
len(df.columns)

19

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [6]:
df.dtypes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

What's the standard deviation of the trips duration in January?

In [7]:
arr = np.array(df.duration)
float(round(arr.std(), 2))

42.59

# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [8]:
float(round(((df.duration >= 1) & (df.duration <= 60)).mean(), 2) * 100)

98.0

# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

In [9]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_18247/1299758185.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


Fit a dictionary vectorizer

In [10]:
dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()

Get a feature matrix from it

In [ ]:
feature_matrix = dv.fit_transform(dicts).toarray()
feature_matrix

What's the dimensionality of this matrix (number of columns)?